In [1]:
# 汎用ライブラリのimport
import sys
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
import tensorflow as tf
import collections
from datetime import datetime
import csv
from itertools import chain

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import scipy.stats

# bayesian
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from scipy.stats import rankdata
from sklearn import metrics
import warnings

In [2]:
# keras用ライブラリ
from keras.datasets import mnist
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
# 自作ライブラリのimport
if os.environ['BR_HOME']+"/boatrace" not in sys.path:
    sys.path.append(os.environ['BR_HOME']+"/boatrace")
#print(sys.path)

from setup.myUtil import dbHandler

In [4]:
# 舟券の配列を取得
funakenList=[]
with open(os.environ['BR_HOME']+'/boatrace/config/3t_list.dat') as f:
    reader=csv.reader(f)
    for row in reader:
        funakenList.append(row)
funakenID = [i for i in range(120)]
funakenDict=dict(zip(funakenList[0],funakenID))

In [5]:
# 分析期間の指定は一旦ここでまとめてみる。
simStartDate="20180101"
simEndDate="20181231"

In [6]:
dbh=dbHandler.getDBHandle()
#dbHandler.closeDBHandle(dbh)

In [7]:
# データを取得
with dbh.cursor() as cursor:
    sel_sql = "select * from raceabst_forml_rentai_v \
               where raceDate between '%s' and '%s' \
               order by raceId "\
               % (simStartDate,simEndDate)
    cursor.execute(sel_sql)
    loadList=cursor.fetchall()
print("simdata:",len(loadList))

simdata: 54083


In [8]:
df = pd.io.json.json_normalize(loadList)
df.head()

,funaken,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1oldavgstdev,l1oldavgsttime,l1oldrank1,l1oldrank2,...,l6oldrank4,l6oldrank5,l6oldrank6,l6rank,l6score,odds,raceDate,raceId,raceWaveHeight,raceWindSpeed
0,2-1-3,1,0.3636,0.5152,0.3558,0.5092,-0.01513,0.17280,0.19,0.14,...,0.13,0.15,0.14,B2,0.851669,30.9,2018-01-01,20180101-06-01,3,5.0
1,2-1-6,0,0.2462,0.4462,0.3466,0.4830,0.02080,0.21640,0.08,0.28,...,0.20,0.17,0.13,B1,0.886552,26.2,2018-01-01,20180101-06-02,3,5.0
2,3-5-4,0,0.3692,0.5231,0.3135,0.4108,0.00215,0.19652,0.03,0.05,...,0.11,0.15,0.24,B1,0.580764,99.2,2018-01-01,20180101-06-03,3,5.0
3,1-5-2,0,0.4308,0.6308,0.3676,0.5514,-0.01063,0.17090,0.26,0.27,...,0.20,0.13,0.14,B1,0.583266,16.8,2018-01-01,20180101-06-04,3,5.0
4,4-1-3,0,0.3636,0.5152,0.4469,0.5754,0.00412,0.17030,0.19,0.18,...,0.06,0.09,0.07,B2,0.470489,51.5,2018-01-01,20180101-06-05,3,5.0


In [ ]:
# 入力のデータ整形
xdf=df.drop(['funaken','odds','raceId','raceDate'],axis=1)
# オッズから作ったスコアは効きすぎるので捨ててみる
xdf=xdf.drop(['l1score','l2score','l3score','l4score','l5score','l6score'],axis=1)
#xdf=xdf.drop(['l1Fcnt','l2Fcnt','l3Fcnt','l4Fcnt','l5Fcnt','l6Fcnt'],axis=1)
#xdf=xdf.drop(['l1oldavgstdev','l2oldavgstdev','l3oldavgstdev','l4oldavgstdev','l5oldavgstdev','l6oldavgstdev'],axis=1)

rankLabel=LabelEncoder()
rankLabel=rankLabel.fit(xdf['l1rank'])
xdf['l1rank']=rankLabel.transform(xdf['l1rank'])
xdf['l2rank']=rankLabel.transform(xdf['l2rank'])
xdf['l3rank']=rankLabel.transform(xdf['l3rank'])
xdf['l4rank']=rankLabel.transform(xdf['l4rank'])
xdf['l5rank']=rankLabel.transform(xdf['l5rank'])
xdf['l6rank']=rankLabel.transform(xdf['l6rank'])
xdf.head()

,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1oldavgstdev,l1oldavgsttime,l1oldrank1,l1oldrank2,l1oldrank3,...,l6oldavgsttime,l6oldrank1,l6oldrank2,l6oldrank3,l6oldrank4,l6oldrank5,l6oldrank6,l6rank,raceWaveHeight,raceWindSpeed
0,1,0.3636,0.5152,0.3558,0.5092,-0.01513,0.17280,0.19,0.14,0.15,...,0.2102,0.01,0.03,0.05,0.13,0.15,0.14,3,3,5.0
1,0,0.2462,0.4462,0.3466,0.4830,0.02080,0.21640,0.08,0.28,0.21,...,0.1608,0.08,0.15,0.27,0.20,0.17,0.13,2,3,5.0
2,0,0.3692,0.5231,0.3135,0.4108,0.00215,0.19652,0.03,0.05,0.05,...,0.3131,0.13,0.23,0.14,0.11,0.15,0.24,2,3,5.0
3,0,0.4308,0.6308,0.3676,0.5514,-0.01063,0.17090,0.26,0.27,0.14,...,0.1769,0.13,0.17,0.23,0.20,0.13,0.14,2,3,5.0
4,0,0.3636,0.5152,0.4469,0.5754,0.00412,0.17030,0.19,0.18,0.12,...,0.1887,0.00,0.01,0.00,0.06,0.09,0.07,3,3,5.0


In [ ]:
# validation向けに平均値、標準偏差を出しておく
zscore_param=xdf.describe()
# 変換の方法はこちら。
print( (xdf.loc["0":"1",["l1boat2r"] ]-zscore_param['l1boat2r']['mean'])/zscore_param['l1boat2r']['std'] )

   l1boat2r
0  0.385434
1 -0.702883


In [ ]:
# normalizeしておく
# ★validationの時に要注意。
xdf=xdf.apply(scipy.stats.zscore, axis=0)

In [ ]:
xdf.head()

,l1Fcnt,l1boat2r,l1boat3r,l1motor2r,l1motor3r,l1oldavgstdev,l1oldavgsttime,l1oldrank1,l1oldrank2,l1oldrank3,...,l6oldavgsttime,l6oldrank1,l6oldrank2,l6oldrank3,l6oldrank4,l6oldrank5,l6oldrank6,l6rank,raceWaveHeight,raceWindSpeed
0,2.383492,0.385438,0.237469,0.274803,0.165048,-0.334426,-0.178330,-0.001551,-0.712480,-0.489666,...,0.492243,-1.355948,-1.626986,-1.592777,-0.458801,-0.234887,-0.344661,1.590847,0.215616,1.162865
1,-0.397120,-0.702890,-0.266202,0.193452,-0.018344,0.602493,0.778055,-1.147828,1.791374,0.795856,...,-0.491896,-0.669148,-0.007846,1.795791,0.702825,0.057451,-0.411413,0.545545,0.215616,1.162865
2,-0.397120,0.437351,0.295136,-0.099231,-0.523721,0.116171,0.341979,-1.668863,-2.322101,-2.632204,...,2.542200,-0.178577,1.071580,-0.206545,-0.790694,-0.234887,0.322868,0.545545,0.215616,1.162865
3,-0.397120,1.008399,1.081301,0.379143,0.460434,-0.217083,-0.220007,0.727898,1.612528,-0.703920,...,-0.171154,-0.178577,0.262010,1.179687,0.702825,-0.527226,-0.344661,0.545545,0.215616,1.162865
4,-0.397120,0.385438,0.237469,1.080345,0.628426,0.167541,-0.233168,-0.001551,0.002907,-1.132427,...,0.063923,-1.454063,-1.896843,-2.362906,-1.620427,-1.111903,-0.811931,1.590847,0.215616,1.162865


In [ ]:
# ファイルから作った辞書で変換する
ydf=df['funaken']
ydf=pd.DataFrame(ydf.replace(funakenDict))
ydf['funaken']=ydf['funaken'].astype(int)
#ydf.head()

# ydfはone-hotにしておく。
y=np.eye(120)[ydf['funaken']]
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# 重み付けのため、オッズのリストを作る
odf=df['odds'].values
raceId_df=df['raceId'].values
#odf=pd.DataFrame(df['odds'])
#odf.describe()
print(odf)
print(raceId_df)

[ 30.9  26.2  99.2 ... 267.6  14.4  10.7]
['20180101-06-01' '20180101-06-02' '20180101-06-03' ... '20181231-24-10'
 '20181231-24-11' '20181231-24-12']


In [ ]:
#train/testを分割
X_train, X_test, y_train, y_test,o_train,o_test,raceId_train,raceId_test = train_test_split(xdf, y,odf,raceId_df)
print("X_train,X_test:",len(X_train),len(X_test))
print(type(X_train))
print("y_train,y_test:",len(y_train),len(y_test))
print(type(y_train))
print("o_train,o_test:",len(o_train),len(o_test))
print(type(o_train))
print("raceId_train,raceId_test:",len(raceId_train),len(raceId_test))
print(type(o_train))

X_train,X_test: 40562 13521
<class 'pandas.core.frame.DataFrame'>
y_train,y_test: 40562 13521
<class 'numpy.ndarray'>
o_train,o_test: 40562 13521
<class 'numpy.ndarray'>
raceId_train,raceId_test: 40562 13521
<class 'numpy.ndarray'>


In [ ]:
# class
class my_Net_bayesian:
    
    def __init__(self,in_hidden_units,in_layer_num):
        # 整数パラメータの保証
        in_hidden_units = int(in_hidden_units)
        in_layer_num = int(in_layer_num)
        assert(type(in_hidden_units)== int)
        assert(type(in_layer_num)== int )
    
        self.Epoch = 400
        self.Batch_size = 4000
        self.Verbose = 1 #ログの出力モード切替 : 1 プログレスバーで表示
        self.output_size = 120
        self.optimize = SGD()
        #self.hidden_units = 128
        #self.hidden_units = 256
        self.hidden_units = in_hidden_units
        self.layer_num = in_layer_num
        self.Validation_split = 0.2 #訓練データの中で検証データとして扱う割合
        #self.Reshape = 28 * 28
        
        self.model = Sequential()
 
    def make_net(self):
        self.model.add(Dense(self.hidden_units, input_shape=(86,)))
        self.model.add(Activation("relu"))
        for i in range(self.layer_num):
            self.model.add(Dense(self.hidden_units))
            self.model.add(Activation("relu"))
        self.model.add(Dense(self.output_size))
        self.model.add(Activation("softmax"))
        self.model.summary()
 
    def model_compile(self):
        self.model.compile(loss="categorical_crossentropy", #交差エントロピー誤差関数
                           optimizer=self.optimize,#確率的勾配下降法
                           metrics=["accuracy"])
 
    def make_model(self):
        self.make_net()
        self.model_compile()
 
    def train(self, x, t):
        self.model.fit(x, t, batch_size=self.Batch_size,
                       epochs=self.Epoch,
                       verbose=self.Verbose,
                       validation_split=self.Validation_split)
 
    def score(self, x, t):
        return self.model.evaluate(x, t, verbose=self.Verbose)



In [ ]:
bounds_Net={
    'in_hidden_units':(300,6000),
    'in_layer_num':(1,4)
}

In [ ]:
def doNet(
    in_hidden_units, # 中間層のユニット数
    in_layer_num    # 層の数
):
    net = my_Net_bayesian(in_hidden_units,in_layer_num)
    net.make_model()
    net.train(X_train, y_train)    
    score = net.score(X_test, y_test)
    del net
    
    return(1.0/score[0])
    #print("\nTest loss:", score[0])
    #print("\nTest accuracy:", score[1])

In [ ]:
Net_BO = BayesianOptimization(doNet,bounds_Net)
print(Net_BO.space.keys)


['in_hidden_units', 'in_layer_num']


In [ ]:
init_points = 5
n_iter = 10

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    Net_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)
    
    

W0812 10:13:30.604851 140462071736128 deprecation_wrapper.py:119] From /home/ec2-user/.pyenv/versions/anaconda3-2018.12/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 10:13:30.606257 140462071736128 deprecation_wrapper.py:119] From /home/ec2-user/.pyenv/versions/anaconda3-2018.12/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 10:13:30.608123 140462071736128 deprecation_wrapper.py:119] From /home/ec2-user/.pyenv/versions/anaconda3-2018.12/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



|   iter    |  target   | in_hid... | in_lay... |
-------------------------------------------------


W0812 10:13:30.652529 140462071736128 deprecation_wrapper.py:119] From /home/ec2-user/.pyenv/versions/anaconda3-2018.12/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0812 10:13:30.662033 140462071736128 deprecation_wrapper.py:119] From /home/ec2-user/.pyenv/versions/anaconda3-2018.12/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3888)              338256    
_________________________________________________________________
activation_1 (Activation)    (None, 3888)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 3888)              15120432  
_________________________________________________________________
activation_2 (Activation)    (None, 3888)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 3888)              15120432  
_________________________________________________________________
activation_3 (Activation)    (None, 3888)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 120)               466680    
__________

W0812 10:13:30.768065 140462071736128 deprecation.py:323] From /home/ec2-user/.pyenv/versions/anaconda3-2018.12/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0812 10:13:30.813236 140462071736128 deprecation_wrapper.py:119] From /home/ec2-user/.pyenv/versions/anaconda3-2018.12/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



...

32449/32449 [==============================] - 62s 2ms/step - loss: 3.9653 - acc: 0.0972 - val_loss: 4.0230 - val_acc: 0.0881
Epoch 177/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.9641 - acc: 0.0973 - val_loss: 4.0220 - val_acc: 0.0873
Epoch 178/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.9627 - acc: 0.0967 - val_loss: 4.0209 - val_acc: 0.0875
Epoch 179/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.9615 - acc: 0.0966 - val_loss: 4.0201 - val_acc: 0.0873
Epoch 180/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.9603 - acc: 0.0976 - val_loss: 4.0191 - val_acc: 0.0891
Epoch 181/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.9589 - acc: 0.0973 - val_loss: 4.0180 - val_acc: 0.0873
Epoch 182/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.9577 - acc: 0.0979 - val_loss: 4.0174 - val_acc: 0.0880
Epoch 183/400
32449/32449 [=======

32449/32449 [==============================] - 62s 2ms/step - loss: 3.9002 - acc: 0.1033 - val_loss: 3.9787 - val_acc: 0.0874
Epoch 235/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8991 - acc: 0.1023 - val_loss: 3.9779 - val_acc: 0.0868
Epoch 236/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8982 - acc: 0.1032 - val_loss: 3.9775 - val_acc: 0.0880
Epoch 237/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8972 - acc: 0.1031 - val_loss: 3.9769 - val_acc: 0.0873
Epoch 238/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8963 - acc: 0.1029 - val_loss: 3.9763 - val_acc: 0.0874
Epoch 239/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8953 - acc: 0.1030 - val_loss: 3.9757 - val_acc: 0.0878
Epoch 240/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8944 - acc: 0.1025 - val_loss: 3.9753 - val_acc: 0.0874
Epoch 241/400
32449/32449 [=======

32449/32449 [==============================] - 62s 2ms/step - loss: 3.8508 - acc: 0.1084 - val_loss: 3.9513 - val_acc: 0.0878
Epoch 293/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8500 - acc: 0.1086 - val_loss: 3.9508 - val_acc: 0.0869
Epoch 294/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8493 - acc: 0.1095 - val_loss: 3.9504 - val_acc: 0.0867
Epoch 295/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8486 - acc: 0.1083 - val_loss: 3.9499 - val_acc: 0.0870
Epoch 296/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8478 - acc: 0.1093 - val_loss: 3.9496 - val_acc: 0.0863
Epoch 297/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8472 - acc: 0.1089 - val_loss: 3.9493 - val_acc: 0.0860
Epoch 298/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8465 - acc: 0.1088 - val_loss: 3.9491 - val_acc: 0.0859
Epoch 299/400
32449/32449 [=======

32449/32449 [==============================] - 62s 2ms/step - loss: 3.8129 - acc: 0.1138 - val_loss: 3.9358 - val_acc: 0.0849
Epoch 351/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8123 - acc: 0.1143 - val_loss: 3.9351 - val_acc: 0.0874
Epoch 352/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8117 - acc: 0.1133 - val_loss: 3.9349 - val_acc: 0.0864
Epoch 353/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8111 - acc: 0.1136 - val_loss: 3.9343 - val_acc: 0.0875
Epoch 354/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8105 - acc: 0.1138 - val_loss: 3.9343 - val_acc: 0.0869
Epoch 355/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8100 - acc: 0.1141 - val_loss: 3.9344 - val_acc: 0.0857
Epoch 356/400
32449/32449 [==============================] - 62s 2ms/step - loss: 3.8094 - acc: 0.1139 - val_loss: 3.9341 - val_acc: 0.0859
Epoch 357/400
32449/32449 [=======

Train on 32449 samples, validate on 8113 samples
Epoch 1/400
32449/32449 [==============================] - 178s 5ms/step - loss: 4.7795 - acc: 0.0118 - val_loss: 4.7605 - val_acc: 0.0217
Epoch 2/400
32449/32449 [==============================] - 176s 5ms/step - loss: 4.7489 - acc: 0.0396 - val_loss: 4.7305 - val_acc: 0.0523
Epoch 3/400
28000/32449 [========================>.....] - ETA: 22s - loss: 4.7209 - acc: 0.0615

In [ ]:
#モデルを保存
model_path=os.path.join(os.environ['BR_HOME'],"boatrace/models")
model_modelfile=os.path.join(model_path,'keras_perceptron_model.json')
model_weightfile=os.path.join(model_path,'keras_perceptron_weight.json')

open(model_modelfile,"w").write(net.model.to_json())
net.model.save_weights(model_weightfile)

In [ ]:
# testの回収率をシミュレート
# オッズを見て判断する場合
resAmount=0
buyAmount=0
resCnt=0
buyCnt=0

testPredictList=net.model.predict(X_test,batch_size=len(X_test))

for i in range(len(raceId_test) ):
    raceId=raceId_train[i]
    with dbh.cursor() as cursor:
        sel_sql = "select funaken,odds from raceodds \
                   where oddsType = '3t' \
                   and raceId = '%s' \
                   order by funaken" \
                   % (raceId)

        cursor.execute(sel_sql)
        loadList=pd.DataFrame(cursor.fetchall())
        loadList=pd.DataFrame(loadList.replace(funakenDict))
                
    for j in range(120):
        # y_predの閾値を下げてみる。
        if testPredictList[i][j]> 0.08 and ( (testPredictList[i][j] * (loadList[loadList['funaken']==j]['odds'])).values[0] > 1.5) :
            print("buy:",raceId,i,j,loadList[loadList['funaken']==j]['odds'].values[0],round(testPredictList[i][j],3) )
            buyAmount+=1
            buyCnt+=1
            if list(y_test[i]).index(1)==j:
                print("☆hit!☆:",raceId,j,loadList[loadList['funaken']==j]['odds'].values[0])
                resAmount += o_test[i]
                resCnt+=1
            else:
                pass
#res=net.model.predict_on_batch(xdf[1:3])


In [ ]:
print("resultReturn:",resAmount/buyAmount)
print("totalRace,buy,return",len(y_test),buyAmount,resAmount )